In [1]:
import cv2

In [2]:
vs=cv2.VideoCapture(2)

In [3]:

# Start webcam (0 is the default camera)
vs = cv2.VideoCapture(0)

while True:
    success, img = vs.read()
    if not success:
        print("Failed to grab frame")
        break

    cv2.imshow("Video Stream", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# Clean up
vs.release()
cv2.destroyAllWindows()


In [4]:

# Start video capture
vs = cv2.VideoCapture(0)

# Read the first frame
ret, prev_frame = vs.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_frame = cv2.GaussianBlur(prev_frame, (21, 21), 0)

while True:
    ret, frame = vs.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Find the absolute difference
    diff = cv2.absdiff(prev_frame, gray)
    thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False

    for contour in contours:
        if cv2.contourArea(contour) < 500:
            continue
        motion_detected = True
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show status
    status_text = "Motion Detected" if motion_detected else "Normal"
    color = (0, 0, 255) if motion_detected else (0, 255, 0)
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Show result
    cv2.imshow("Motion Detection", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): # press q for exit
        break

    prev_frame = gray

# Cleanup
vs.release()
cv2.destroyAllWindows()
